In [1]:
from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn import preprocessing

In [3]:
def costFunctionBatteryLifetime2(x,demand,pv_gen,bat_t0,elprices,battery):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    dim = len(x)
    bat_action = x[0]-bat_t0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == battery.capacity:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == 0.0:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == battery.capacity:
            charOrDisch=-1
        elif x[0] == 0.0:
            charOrDisch=1
            
    #grid_sum=(demand[0]-pv_gen[0]+bat_action)*elprices[0]
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == battery.capacity:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == 0.0:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == battery.capacity:
                charOrDisch=-1
            elif x[i] == 0.0:
                charOrDisch=1
        
        #grid_sum+=((demand[i]-pv_gen[i]+bat_action)*elprices[i])
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = (grid_sum)-10*(contChar+contDisch)
    #fitness = -(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

In [4]:
class Battery:
    def __init__(self,cap,soc,act,contAct,numContChar,numContDisch):
        self.capacity=cap
        self.SOC=soc
        self.action=act
        self.contAct=contAct
        self.numContChar=numContChar
        self.numContDisch=numContDisch

In [5]:
class BatAlgorithm2():
    def __init__(self, D, NP, N_Gen, A, r, Qmin, Qmax, Lower, Upper, function,demand,pv_gen,bat_t0,elprices, battery):
        self.D = D  #dimension
        self.NP = NP  #population size 
        self.N_Gen = N_Gen  #generations
        self.A = A  #loudness
        self.r = r  #pulse rate
        self.Qmin = Qmin  #frequency min
        self.Qmax = Qmax  #frequency max
        self.Lower = Lower  #lower bound
        self.Upper = Upper  #upper bound
        self.battery = battery#pridal

        self.f_min = 0.0  #minimum fitness
        self.charOrDischBest = -2   #pridal
        self.contCharBest = -2      #pridal
        self.contDischBest = -2     #pridal
        self.gridPrice = 0
        
        self.Lb = [0] * self.D  #lower bound
        self.Ub = [0] * self.D  #upper bound
        self.Q = [0] * self.NP  #frequency

        self.v = [[0 for i in range(self.D)] for j in range(self.NP)]  #velocity
        self.Sol = [[0 for i in range(self.D)] for j in range(self.NP)]  #population of solutions
        self.Fitness = [0] * self.NP  #fitness
        self.best = [0] * self.D  #best solution
        self.Fun = function
        
        self.charOrDisch = [0] * self.NP #pridal
        self.contChar = [0] * self.NP    #pridal
        self.contDisch = [0] * self.NP   #pridal

        
        self.demand = demand  #som pridal
        self.pv_gen = pv_gen  #som pridal
        self.elprices = elprices   #som pridal
        self.bat_t0 = bat_t0  #som pridal
        self.grid_actions = []#som pridal
        self.demand_sum = 0   #som pridal
        self.grid_total = 0   #som pridal
        #print(demand)
        #print(pv_gen)
        #print(bat_t0)


    def best_bat(self):
        i = 0
        j = 0
        for i in range(self.NP):
            if self.Fitness[i] < self.Fitness[j]:
                j = i
        for i in range(self.D):
            self.best[i] = self.Sol[j][i]
        self.f_min = self.Fitness[j]

    def init_bat(self):
        for i in range(self.D):
            self.Lb[i] = self.Lower
            self.Ub[i] = self.Upper

        for i in range(self.NP):
            self.Q[i] = 0
            for j in range(self.D):
                rnd = np.random.uniform(0, 1)
                self.v[i][j] = 0.0
                self.Sol[i][j] = self.Lb[j] + (self.Ub[j] - self.Lb[j]) * rnd
            #self.Fitness[i] = self.Fun(self.D, self.Sol[i])                                    #som zmenil
            self.Fitness[i], self.charOrDisch, self.contChar, self.contDisch, self.gridPrice = self.Fun(self.Sol[i],self.demand,self.pv_gen,self.bat_t0,self.elprices, self.battery)         #som zmenil
        self.best_bat()

    def simplebounds(self, val, lower, upper):
        if val < lower:
            val = lower
        if val > upper:
            val = upper
        return val

    def move_bat(self):
        S = [[0.0 for i in range(self.D)] for j in range(self.NP)]

        self.init_bat()

        for t in range(self.N_Gen):
            for i in range(self.NP):
                rnd = np.random.uniform(0, 1)
                self.Q[i] = self.Qmin + (self.Qmax - self.Qmin) * rnd
                for j in range(self.D):
                    self.v[i][j] = self.v[i][j] + (self.Sol[i][j] -
                                                   self.best[j]) * self.Q[i]
                    S[i][j] = self.Sol[i][j] + self.v[i][j]

                    S[i][j] = self.simplebounds(S[i][j], self.Lb[j],
                                                self.Ub[j])

                rnd = np.random.random_sample()

                if rnd > self.r:
                    for j in range(self.D):
                        S[i][j] = self.best[j] + 0.001 * random.gauss(0, 1)
                        S[i][j] = self.simplebounds(S[i][j], self.Lb[j],
                                                self.Ub[j])
                        
                #Fnew = self.Fun(self.D, S[i])                                           #som zmenil
                Fnew, charOrDischNew, contCharNew, contDischNew, gridPriceNew = self.Fun(S[i],self.demand,self.pv_gen,self.bat_t0,self.elprices, self.battery)                #som zmenil

                rnd = np.random.random_sample()

                if (Fnew <= self.Fitness[i]) and (rnd < self.A):
                    for j in range(self.D):
                        self.Sol[i][j] = S[i][j]
                    self.Fitness[i] = Fnew

                if Fnew <= self.f_min:
                    for j in range(self.D):
                        self.best[j] = S[i][j]
                    self.f_min = Fnew
                    self.charOrDischBest = charOrDischNew
                    self.contCharBest = contCharNew
                    self.contDischBest = contDischNew
                    self.gridPrice = gridPriceNew

        ##som pridal
        #bat_actions = []                                                #som pridal
        #self.grid_actions = []
        #bat_actions.append(self.best[0]-self.bat_t0)
        #self.grid_actions.append(self.demand[0]-self.pv_gen[0]+bat_actions[0])
        ##pv_actions.append()
        #self.demand_sum = self.demand[0]
        #self.grid_total = self.grid_actions[0]
        #for i in range(1,self.D):
         #   bat_actions.append(self.best[i]-self.best[i-1])
         #   self.grid_actions.append(self.demand[i]-self.pv_gen[i]+bat_actions[i])
         #   self.demand_sum+=self.demand[i]
         #   self.grid_total+=self.grid_actions[i]
        ##print('cena: ',self.f_min)
        ##print(self.best)
        ##bat.SOC=self.best[self.D-1]
        ##print()

In [6]:
class BAOptimizer():
    def __init__(self):
        #data = pd.read_csv('ausgrid/customer_1.csv', index_col=0, sep=";")
        data = pd.read_csv('../data/load/dataid171.csv', index_col=0, sep=";")
        data.loc[data['loadProd'] < 0, 'loadProd'] = 0
        print(data.head())
        
        self.gen = pd.DataFrame(data[['loadProd']])
        self.con = pd.DataFrame(data[['loadCons']])
        self.con = self.con['loadCons']
        self.gen = self.gen['loadProd']
        
    def optimize(self):
        windowSize=10
        start=3626
        #end=18*7
        end = 3700#761#4345
        bat=Battery(13, 0, 1, 0, 0, 0)
        total_demand = 0
        total_grid = 0
        total_price = 0

        gridTotalProgress = []
        batSOCProgress = []
        batSOCProgress.append(0)

        for c in range(start,end,windowSize):
            d=c+windowSize
            initial= []
            genWindow = []
            conWindow = []
            BSSBounds = []
            elprices = []
            for i in range(c,d):
                if i<=end:
                    hour = i-start
                    hour = hour % 24
                    if hour >=0 and hour <= 6:
                        elprices.append(0.005)
                    if hour >=7 and hour <= 14:
                        elprices.append(0.057)
                    if hour >=15 and hour <= 20:
                        elprices.append(0.09)
                    if hour >=21 and hour <= 22:
                        elprices.append(0.057)
                    if hour >=23 and hour <= 24:
                        elprices.append(0.005)
                    genWindow.append(self.gen[i])
                    conWindow.append(self.con[i])
                    BSSBounds.append((0,bat.capacity))
                    if i % 5 == 0:
                        print(i)     
            Algorithm = BatAlgorithm2(len(genWindow), 10000, 200, 0.5, 0.5, 0.0, 2.0, 0.0, bat.capacity, costFunctionBatteryLifetime2, conWindow, genWindow, bat.SOC, elprices, bat)
            Algorithm.move_bat()
            
            #prerobene
            alg_bat_actions = []                                                
            alg_grid_actions = []
            alg_bat_actions.append(Algorithm.best[0]-bat.SOC)
            alg_grid_actions.append(conWindow[0]-genWindow[0]+alg_bat_actions[0])
            alg_demand_sum = conWindow[0]
            alg_grid_total = alg_grid_actions[0]
            for k in range(1,Algorithm.D):
                alg_bat_actions.append(Algorithm.best[k]-Algorithm.best[k-1])
                alg_grid_actions.append(conWindow[k]-genWindow[k]+alg_bat_actions[k])
                alg_demand_sum+=conWindow[k]
                alg_grid_total+=alg_grid_actions[k]
            #prerobene
            
            bat.SOC=Algorithm.best[Algorithm.D-1]
            print(conWindow, genWindow)
            l = len(alg_grid_actions)
            for j in range(0,l):
                gridTotalProgress.append(alg_grid_actions[j])
                batSOCProgress.append(Algorithm.best[j])
            total_demand += alg_demand_sum#prerobene
            total_grid += alg_grid_total#prerobene
            total_price += Algorithm.gridPrice
            bat.contAct=Algorithm.charOrDischBest
            bat.numContChar += Algorithm.contCharBest
            bat.numContDisch += Algorithm.contDischBest
            print("el price:",Algorithm.gridPrice)
            print("el prices:",elprices)
            print("grid actions:",alg_grid_actions)#prerobene
            print("battery soc:",Algorithm.best)
            print("battery contAct:",bat.contAct)
            print("battery numContChar:",bat.numContChar)
            print("battery numContDisch:",bat.numContDisch)

        print('FINAL:')
        print('Total demand: ',total_demand)
        print('Total grid: ',total_grid)
        print('Total price: ',total_price)
                
            #print(c, genWindow)
            #psoOpt = PSO(cost_function,initial,BSSBounds,num_particles=1000,maxiter=100,demand=conWindow,pv_gen=genWindow,bat_t0=bat.SOC)
            #l = len(psoOpt.grid_actions)
        g=0
        gi=0
        ge=0

        b=0
        bc=0
        bd=0

        for i in(gridTotalProgress):
            if i >= 0:
                gi=gi+i
            if i < 0:
                ge=ge-i
            g=g+i

        l = len(batSOCProgress)
        print('len ', l)
        for i in range(1,l):
            action = batSOCProgress[i]-batSOCProgress[i-1]
            if action > 0:
                bc=bc+action
            if action <= 0:
                bd=bd-action
            b=b+action
        
        print(g)
        print(gi)
        print(ge)

        print()

        #print(batSOCProgress)
        print(b)
        print(bc)
        print(bd)#111.2 85.49

In [7]:
ba = BAOptimizer()
ba.optimize()


          date      time             dateTime  dayOfWeek  holiday  dayId  \
id                                                                         
1   2014-01-01  00:00:00  2014-01-01 00:00:00          3        1      1   
2   2014-01-01  01:00:00  2014-01-01 01:00:00          3        1      1   
3   2014-01-01  02:00:00  2014-01-01 02:00:00          3        1      1   
4   2014-01-01  03:00:00  2014-01-01 03:00:00          3        1      1   
5   2014-01-01  04:00:00  2014-01-01 04:00:00          3        1      1   

    loadCons  loadProd  loadGrid  
id                                
1   0.885267       0.0  0.885267  
2   0.512767       0.0  0.512767  
3   0.448867       0.0  0.448867  
4   0.473483       0.0  0.473483  
5   0.440150       0.0  0.440150  
3630
3635
[0.663233333333333, 0.618716666666667, 0.45806666666666707, 0.4382333333333329, 0.54785, 0.645216666666667, 0.4785833333333329, 0.74405, 0.815316666666667, 0.9213] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.229866666666666